<a href="https://colab.research.google.com/github/li2he1/Cota-Healthcare-Data-Analysis/blob/main/Cota_Healthcare.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


# Cota Healthcare Data Analysis


# Part 0: Setup Google Drive Environment

In [432]:
#@title import packages
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')
# will show all the columns
pd.set_option('display.max_columns', None)

In [433]:
#Load CSV files
url = 'https://raw.githubusercontent.com/li2he1/Cota-Healthcare-Data-Analysis/main/cancer-datasets_filtered_pancan_clinical.csv'
df1 = pd.read_csv(url)


# Part 1: Data Exploration



In [434]:
#df1

1.How many columns are in this dataset?

In [435]:
print ("Num of rows: " + str(df1.shape[0])) # row count
print ("Num of columns: " + str(df1.shape[1])) # col count

Num of rows: 10761
Num of columns: 746


In [436]:
#!pip uninstall pandas_profiling
#!pip install -U pandas_profiling

In [437]:
# import pandas_profiling
# pandas_profiling.ProfileReport(df1)

2.Breakdown columns by datatypes with counts

In [438]:
df1.dtypes.value_counts()

object     500
float64    246
dtype: int64

3.Breakdown counts by gender, race and ethnicity

In [439]:
df1.gender.value_counts()

FEMALE    5623
MALE      5138
Name: gender, dtype: int64

In [440]:
df1.race.value_counts()

WHITE                                        7862
BLACK OR AFRICAN AMERICAN                     902
ASIAN                                         670
[Not Evaluated]                               153
[Unknown]                                     131
AMERICAN INDIAN OR ALASKA NATIVE               27
NATIVE HAWAIIAN OR OTHER PACIFIC ISLANDER      13
Name: race, dtype: int64

In [441]:
df1.ethnicity.value_counts()

NOT HISPANIC OR LATINO    7850
[Not Evaluated]            573
HISPANIC OR LATINO         366
[Unknown]                  186
Name: ethnicity, dtype: int64

4.Report of missingness across all columns

In [442]:
df1.isnull().any(axis=0) # check if there is a NaN in a column

bcr_patient_uuid                   False
bcr_patient_barcode                False
acronym                            False
gender                             False
vital_status                        True
                                   ...  
fetoprotein_outcome_lower_limit     True
inter_norm_ratio_lower_limit        True
family_cancer_type_txt              True
bilirubin_upper_limit               True
days_to_last_known_alive            True
Length: 746, dtype: bool

In [443]:
df1.notnull().all(axis=0) # check the colums that has no NaN

bcr_patient_uuid                    True
bcr_patient_barcode                 True
acronym                             True
gender                              True
vital_status                       False
                                   ...  
fetoprotein_outcome_lower_limit    False
inter_norm_ratio_lower_limit       False
family_cancer_type_txt             False
bilirubin_upper_limit              False
days_to_last_known_alive           False
Length: 746, dtype: bool

In [444]:
df1.isnull().sum()

bcr_patient_uuid                       0
bcr_patient_barcode                    0
acronym                                0
gender                                 0
vital_status                           4
                                   ...  
fetoprotein_outcome_lower_limit    10736
inter_norm_ratio_lower_limit       10734
family_cancer_type_txt             10740
bilirubin_upper_limit              10729
days_to_last_known_alive           10750
Length: 746, dtype: int64

In [445]:
df1.isnull().sum()/len(df1)*100

bcr_patient_uuid                    0.000000
bcr_patient_barcode                 0.000000
acronym                             0.000000
gender                              0.000000
vital_status                        0.037171
                                     ...    
fetoprotein_outcome_lower_limit    99.767680
inter_norm_ratio_lower_limit       99.749094
family_cancer_type_txt             99.804851
bilirubin_upper_limit              99.702630
days_to_last_known_alive           99.897779
Length: 746, dtype: float64

In [446]:
df1.info(verbose=True, null_counts=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10761 entries, 0 to 10760
Data columns (total 746 columns):
 #   Column                                                                                                            Non-Null Count  Dtype  
---  ------                                                                                                            --------------  -----  
 0   bcr_patient_uuid                                                                                                  10761 non-null  object 
 1   bcr_patient_barcode                                                                                               10761 non-null  object 
 2   acronym                                                                                                           10761 non-null  object 
 3   gender                                                                                                            10761 non-null  object 
 4   vital_status               

In [447]:
# Delete columns containing either 99% or more than 99% NaN Values
perc = 99.0
min_count =  int(((100-perc)/100)*df1.shape[0] + 1)
mod_df = df1.dropna( axis=1, 
                thresh=min_count)

5.What’s the average age of a patient in this dataset?

---



In [448]:
# The challenge is to know when is the end date for calculating the age and the survival time in question 9. 
# Also , we need to consider the days to death, if we know the meaning of day to birth and day to death, then we can make it more accuratly
#In this question, I assume today is the end date for calculating the age.


In [449]:
import datetime
dt = datetime.datetime.today()
df2 = df1.dropna(subset=['days_to_birth'])

In [450]:
df2['delta_of_birth'] = df2.apply(lambda row : datetime.timedelta(days = -row.days_to_birth), axis = 1) 

In [451]:
df2['year_of_birth'] = df2.apply(lambda row : (dt- row.delta_of_birth).year, axis = 1) 

consider death days

In [452]:
df2["days_to_death"]=df2["days_to_death"].astype(str)

In [453]:
df2["days_to_death"]=df2["days_to_death"].replace({'[Not Applicable]':0,'nan': 0,"[Discrepancy]":0})

In [454]:
df2["days_to_death"]=pd.to_numeric(df2["days_to_death"])

In [455]:
df2['delta_of_death'] = df2.apply(lambda row : datetime.timedelta(days = row.days_to_death), axis = 1) 

In [456]:
df2['year_of_end'] = df2.apply(lambda row : (dt- row.delta_of_death).year, axis = 1) 

In [457]:
df2['age'] = df2.apply(lambda row : row.year_of_end- row.year_of_birth, axis = 1) 

In [458]:
df2['age'].mean()

58.810409620443444

6.Breakdown of the patient’s birth year

In [459]:
df2['year_of_birth']

0        1959
1        1966
2        1957
3        1967
4        1931
         ... 
10756    1964
10757    1940
10758    1976
10759    1940
10760    1964
Name: year_of_birth, Length: 10644, dtype: int64

In [460]:
df2['year_of_birth'].describe()

count    10644.000000
mean      1961.452461
std         14.378545
min       1931.000000
25%       1951.000000
50%       1960.000000
75%       1970.000000
max       2010.000000
Name: year_of_birth, dtype: float64

7.Provide a list of the top 20 sorted values of the column - Eastern Cancer Oncology Group. Make assumptions as needed. Hint: You may have to normalize the dataset to do this.

Sorry I am confused about this question because I don't have meta data about the meaning of each column. There are 11 unique value in this column and I don't know what is the meaning by top 20? I may need more information about this question.

I found some information about Performance Status Assessment by Using ECOG includes 5 values from 0 to 5 according to https://www.karger.com/Article/FullText/503095 which is used for assessing patient's oncology status. I can normalized it for further prediction or machine learning use.

Hope I can talk to you and expore the data with your guidance and advice. Thank you very much.

In [461]:
df1.eastern_cancer_oncology_group.isnull().sum()

8455

In [462]:
df1.eastern_cancer_oncology_group.describe()

count     2306
unique      11
top          0
freq       778
Name: eastern_cancer_oncology_group, dtype: object

In [463]:
df1.eastern_cancer_oncology_group.value_counts()

0                  778
1                  642
[Not Evaluated]    366
[Unknown]          221
2                  133
0.0                 60
1.0                 50
3                   39
4                    9
2.0                  7
4.0                  1
Name: eastern_cancer_oncology_group, dtype: int64

In [464]:
df3 = df1.dropna(subset=['eastern_cancer_oncology_group'])

In [465]:
df3 = df3.loc[(df3['eastern_cancer_oncology_group'] != '[Not Evaluated]') & (df3['eastern_cancer_oncology_group'] != '[Unknown]') ]

In [466]:
df3['eastern_cancer_oncology_group'] = df3['eastern_cancer_oncology_group'].astype(float)

In [467]:
df3.eastern_cancer_oncology_group.value_counts()

0.0    838
1.0    692
2.0    140
3.0     39
4.0     10
Name: eastern_cancer_oncology_group, dtype: int64

In [468]:
#min-max feature scaling
  
# apply normalization techniques by Column 1
column = 'eastern_cancer_oncology_group'
df3['eastern_cancer_oncology_group_scale'] = (df3[column] - df3[column].min()) / (df3[column].max() - df3[column].min())    
  

In [469]:
df3['eastern_cancer_oncology_group_scale'].value_counts()

0.00    838
0.25    692
0.50    140
0.75     39
1.00     10
Name: eastern_cancer_oncology_group_scale, dtype: int64

8.Identify Metastatic Cancer patients, and group them by ICD codes

In [470]:
#find Metastatic columns
Metastatic_cols = [col for col in mod_df.columns if 'metastatic' in col]
print(Metastatic_cols)

['malignant_neoplasm_metastatic_involvement_site', 'other_metastatic_involvement_anatomic_site', 'metastatic_site_at_diagnosis_other', 'metastatic_neoplasm_confirmed']


In [471]:
df1[Metastatic_cols].isnull().sum()/len(df1)*100

malignant_neoplasm_metastatic_involvement_site    97.927702
other_metastatic_involvement_anatomic_site        95.260664
metastatic_site_at_diagnosis_other                97.611746
metastatic_neoplasm_confirmed                     98.290122
dtype: float64

In [472]:
#find ICD columns
ICD_cols = [col for col in mod_df.columns if 'icd' in col]
print(ICD_cols)

['icd_10', 'icd_o_3_histology', 'icd_o_3_site', 'cancer_diagnosis_cancer_type_icd9_text_name']


In [473]:
df1[ICD_cols].isnull().sum()/len(df1)*100

icd_10                                          0.009293
icd_o_3_histology                               0.009293
icd_o_3_site                                    0.009293
cancer_diagnosis_cancer_type_icd9_text_name    98.606078
dtype: float64

In [474]:
df1.icd_10.unique()

array(['C02.9', 'C15.4', 'C15.5', 'C16.0', 'C18.0', 'C18.2', 'C18.7',
       'C19', 'C20', 'C34.1', 'C34.3', 'C34.12', 'C34.9', 'C48.0',
       'C48.1', 'C48.2', 'C49.5', 'C50.9', 'C56.9', 'C61', 'C63.1',
       'C64.9', 'C64.1', 'C67.9', 'C71.0', 'C71.2', 'C71.9', 'C74.9', nan,
       'C00.9', 'C01', 'C03.9', 'C04.0', 'C04.9', 'C05.9', 'C06.0',
       'C06.2', 'C06.9', 'C07', 'C08.0', 'C09.9', 'C10.3', 'C10.9',
       'C13.9', 'C14.8', 'C15.9', 'C15.3', 'C16.1', 'C16.2', 'C16.3',
       'C83.3', 'C16.9', 'C17.9', 'C18.3', 'C18.4', 'C18.5', 'C18.6',
       'C18.9', 'C43.51', 'C22.0', 'C22.1', 'C25.0', 'C25.1', 'C25.2',
       'C25.9', 'C44.31', 'C32.1', 'C32.9', 'C34.0', 'C34.2', 'C34.30',
       'C34.8', 'C37', 'C38.1', 'C45.0', 'C44.2', 'C44.3', 'C44.4',
       'C44.5', 'C44.50', 'C44.6', 'C44.601', 'C44.7', 'C44.701', 'C44.9',
       'C49.0', 'C49.10', 'C49.1', 'C49.2', 'C49.20', 'C49.3', 'C49.4',
       'C49.6', 'C49.9', 'C50.2', 'C51.9', 'C52', 'C53.0', 'C53.1',
       'C53.9', 'C

In [475]:
#Metastatic cancer patients are patients who have  have a clinical OR pathologic M value which has ‘M1’ in it.
clinical_cols = [col for col in mod_df.columns if 'clinical' in col]
print(clinical_cols)


['clinical_M', 'clinical_stage', 'clinical_T', 'clinical_N']


In [476]:
df1['clinical_M'].isnull().sum()/len(df1)*100

20.825202118762196

In [477]:
df1['clinical_M'].unique()

array(['M0', 'M1', nan, '[Not Applicable]', 'M1b', 'MX', 'M1a', 'M1c'],
      dtype=object)

In [478]:
pathologic_cols = [col for col in mod_df.columns if 'pathologic' in col]
print(pathologic_cols)

['days_to_initial_pathologic_diagnosis', 'age_at_initial_pathologic_diagnosis', 'year_of_initial_pathologic_diagnosis', 'pathologic_T', 'pathologic_M', 'pathologic_N', 'pathologic_stage', 'initial_pathologic_diagnosis_method', 'presence_of_pathological_nodal_extracapsular_spread', 'cervical_neoplasm_pathologic_margin_involved_text', 'pathologic_tumor_width', 'pathologic_tumor_length', 'pathologic_tumor_depth']


In [479]:
df1['pathologic_M'].isnull().sum()/len(df1)*100

18.344020072483968

In [480]:
df1['pathologic_M'].unique()

array(['M0', 'MX', 'M1', nan, '[Not Applicable]', 'M1b', 'M1a', 'M1c',
       'cM0 (i+)', '[Unknown]'], dtype=object)

In [481]:
df5 = df1.loc[df1['pathologic_M'].str.contains("M1") | df1['clinical_M'].str.contains("M1")]

In [482]:
df5[['clinical_M','pathologic_M']]

,clinical_M,pathologic_M
1,M1,M0
17,[Not Applicable],M1
626,M1b,[Not Applicable]
628,NaN,M1
634,NaN,M1
...,...,...
10413,M1,[Not Applicable]
10681,[Not Applicable],M1c
10690,[Not Applicable],M1c
10711,[Not Applicable],M1c


In [483]:
#Metastatic cancer patients are patients who have stage 4 or above 
#I assume it is the ECOG status 4
df4 = df3.loc[df3['eastern_cancer_oncology_group']>=4]

In [484]:
df4.eastern_cancer_oncology_group.value_counts

<bound method IndexOpsMixin.value_counts of 725     4.0
782     4.0
1922    4.0
1943    4.0
3384    4.0
3385    4.0
3845    4.0
5394    4.0
7632    4.0
9911    4.0
Name: eastern_cancer_oncology_group, dtype: float64>

In [485]:
df6 = pd.concat([df4,df5]).drop_duplicates().reset_index(drop=True)

In [486]:
df6[['eastern_cancer_oncology_group','clinical_M','pathologic_M']]

,eastern_cancer_oncology_group,clinical_M,pathologic_M
0,4,NaN,NaN
1,4,NaN,NaN
2,4,[Not Applicable],M0
3,4,[Not Applicable],M0
4,4,[Not Applicable],[Not Applicable]
...,...,...,...
377,NaN,M1,[Not Applicable]
378,NaN,[Not Applicable],M1c
379,NaN,[Not Applicable],M1c
380,NaN,[Not Applicable],M1c


In [487]:
#group them by ICD codes

In [488]:
df6.dropna(subset=['icd_10'], inplace= True)

In [489]:
grouped_df = df6.groupby("icd_10")

grouped_lists = grouped_df["patient_id"].apply(list)

grouped_lists = grouped_lists.reset_index()

In [490]:
print(grouped_lists)

   icd_10                                         patient_id
0   C02.9                                             [A6SH]
1   C04.9                                             [A92N]
2   C15.3                                             [A51D]
3   C15.4                                 [A8EU, AA7B, A8EX]
4   C15.5               [A5U5, A4OT, A8NF, A8W5, A8NH, A9GO]
..    ...                                                ...
56    C73  [A4ET, A0Y3, A3PA, A0Z2, A4UR, A3GY, A3MY, A3M...
57  C74.0  [A5PI, A5OH, A5LB, A5JG, A5LI, A5JM, A5LJ, A5J...
58  C77.3                                       [A8ZT, AAP0]
59  C77.4                                       [A5GS, A51N]
60  C77.9                                       [A1ZH, A1Q8]

[61 rows x 2 columns]


9. Provide a survival analysis of all Breast Cancer patients.

In [491]:
#!pip install survival

In [492]:
#pip install rpy2
# since library survminer is not avaliable in python, I can use rpy2 to run R library in python
# Or I can use scikit-survival, KaplanMeierFitter to do the survival analysis, but its visulization is not as good as R package 

In [493]:
#According to the informationonSurvivalAnalysis>[http://www.sthda.com/english/wiki/survival- analysis-basics]
#I try to select the sample columns that can be analysis according to the example based on the link provided
 

In [494]:
#find karno columns
karno_cols = [col for col in df1.columns if 'karno' in col]
print(karno_cols)

['karnofsky_performance_score']


In [495]:
#find status columns
status_cols = [col for col in df1.columns if 'status' in col]
print(status_cols)
# I choose vital_status

['vital_status', 'person_neoplasm_cancer_status', 'patient_progression_status', 'performance_status_scale_timing', 'menopause_status', 'margin_status', 'hiv_status', 'breast_carcinoma_progesterone_receptor_status', 'metastatic_breast_carcinoma_progesterone_receptor_status', 'metastatic_breast_carcinoma_lab_proc_her2_neu_immunohistochemistry_receptor_status', 'breast_cancer_surgery_margin_status', 'breast_carcinoma_estrogen_receptor_status', 'lab_proc_her2_neu_immunohistochemistry_receptor_status', 'metastatic_breast_carcinoma_estrogen_receptor_status', 'post_surgical_procedure_assessment_thyroid_gland_carcinoma_status', 'egfr_amplication_status', 'hpv_status_by_ish_testing', 'hpv_status_by_p16_testing', 'mental_status_changes', 'days_to_initial_score_performance_status_scale', 'extrathyroid_carcinoma_present_extension_status', 'bladder_carcinoma_extracapsular_extension_status', 'days_to_performance_status_assessment', 'female_breast_feeding_or_pregnancy_status_indicator', 'performance_

In [496]:
#find time columns
time_cols = [col for col in df1.columns if 'day' in col]
print(time_cols)

['days_to_birth', 'days_to_death', 'days_to_last_followup', 'days_to_initial_pathologic_diagnosis', 'days_to_patient_progression_free', 'days_to_first_response', 'days_to_first_partial_response', 'days_to_first_complete_response', 'days_to_tumor_progression', 'days_to_sample_procurement', 'days_to_hiv_diagnosis', 'days_to_new_tumor_event_after_initial_treatment', 'days_to_diagnostic_mri_performed', 'days_to_diagnostic_computed_tomography_performed', 'amount_of_alcohol_consumption_per_day', 'days_to_initial_score_performance_status_scale', 'days_to_bone_scan_performed', 'days_to_first_biochemical_recurrence', 'days_to_psa', 'interferon_90_day_prior_excision_admin_indicator', 'days_to_submitted_specimen_dx', 'mibc_90day_post_resection_bcg', 'days_to_definitive_surgical_procedure_performed', 'days_to_laboratory_procedure_tumor_marker_squamous_cell_carcinoma_antigen_result', 'days_to_performance_status_assessment', 'days_to_fdg_or_ct_pet_performed', 'days_to_well_differentiated_liposarcoma

In [497]:
# To do the analysis based on the example provided 
# We need to get time: Survival time in days
#The time from ‘response to treatment’ (complete remission) to the occurrence of the event of interest is commonly called survival time (or time to event).

In [498]:
df1.days_to_first_complete_response.isnull().sum()

10761

In [499]:
df1.days_to_initial_pathologic_diagnosis.isnull().sum()

123

In [500]:
df1.days_to_initial_pathologic_diagnosis.describe()

count    10638.0
mean         0.0
std          0.0
min          0.0
25%          0.0
50%          0.0
75%          0.0
max          0.0
Name: days_to_initial_pathologic_diagnosis, dtype: float64

In [501]:
df1.days_to_last_known_alive.isnull().sum()

10750

In [502]:
df1.days_to_death.isnull().sum()
# Should I used days to death and current date to calculate the survival time?

12

I think this data is very interesting, it has 375 colums. I feel understanding  the meaning of colums and how to use it correctly is very interesting and challenging. If I have a chance to talk to you and get any more information or details, guidance or advice, I will definitely work it out.